**Лабораторна робота №2**

Завдання 2.2. Порівняння якості класифікаторів SVM з нелінійними ядрами

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.svm import LinearSVC, SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
#Вхідний файл, який містить дані
input_file = '/content/income_data.txt'

In [3]:
#Читання даних
X = []
y = []
count_class1 = 0
count_class2 = 0
max_datapoints = 25000

In [4]:
with open(input_file, 'r') as f:
    for line in f.readlines():
        if count_class1 >= max_datapoints and count_class2 >= max_datapoints:
            break

        if '?' in line:
            continue

        data = line[:-1].split(',')
        # print(data)

        if data[-1] == ' <=50K' and count_class1 < max_datapoints:
            X.append(data)
            count_class1 += 1

        if data[-1] == ' >50K' and count_class2 < max_datapoints:
            X.append(data)
            count_class2 += 1

In [5]:
# Перетворення на масив numpy
X = np.array(X)

In [6]:
label_encoder = []
X_encoded = np.empty(X.shape)
for i, item in enumerate(X[0]):
    if item.isdigit():
        X_encoded[:, i] = X[:, i]
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        X_encoded[:, i] = label_encoder[-1].fit_transform(X[:, i])

X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int)

In [7]:
#Створення SVМ-класифікаторів
classifiers = {
    'Linear SVM': LinearSVC(random_state=0),
    'RBF SVM': SVC(kernel='rbf', random_state=0),
    'Poly SVM (degree=3)': SVC(kernel='poly', degree=3, random_state=0),
    'Sigmoid SVM': SVC(kernel='sigmoid', random_state=0)
}

#Навчання класифікатора
for classifier in classifiers.items():
    print(f'\nClassifier: {classifier[0]}')
    clf = OneVsRestClassifier(classifier[1])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)

    f1 = f1_score(y_test, y_test_pred, average='weighted')
    precision = precision_score(y_test, y_test_pred, average='weighted', zero_division=0)
    accuracy = accuracy_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred, average='weighted')

    print(f'F1: {f1*100:.2f}%')
    print(f'Precision: {precision*100:.2f}%')
    print(f'Accuracy: {accuracy*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')


Classifier: Linear SVM
F1: 74.32%
Precision: 76.88%
Accuracy: 78.34%
Recall: 78.34%

Classifier: RBF SVM
F1: 63.80%
Precision: 55.71%
Accuracy: 74.64%
Recall: 74.64%

Classifier: Poly SVM (degree=3)
F1: 63.80%
Precision: 55.71%
Accuracy: 74.64%
Recall: 74.64%

Classifier: Sigmoid SVM
F1: 63.68%
Precision: 63.56%
Accuracy: 63.82%
Recall: 63.82%


In [8]:
# Передбачення результату для тестової точки даних
input_data = ['37', ' Private', ' 215646', ' HS-grad', ' 9', ' Never-married', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', ' 0', ' 0', ' 40', ' United-States']


In [9]:
# Кодування тестової точки даних
input_data_encoded = [-1] * len(input_data)
count = 0
for i, item in enumerate(input_data):
    if item.isdigit():
        input_data_encoded[i] = int(item)
    else:
        input_data_encoded[i] = label_encoder[count].transform([item])[0]
        count += 1

input_data_encoded = np.array(input_data_encoded).reshape(1, -1)

In [10]:
for classifier in classifiers.values():
    print(f'\nClassifier: {classifier.__class__.__name__}')
    classifier.fit(X, y)
    predicted_class = classifier.predict(input_data_encoded)
    print(label_encoder[-1].inverse_transform(predicted_class)[0])


Classifier: LinearSVC
 <=50K

Classifier: SVC
 <=50K

Classifier: SVC
 <=50K

Classifier: SVC
 <=50K
